In [17]:
!pip install category_encoders

   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.7 MB 2.1 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/10.7 MB 2.0 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/10.7 MB 2.0 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/10.7 MB 2.1 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/10.7 MB 2.0 MB/s eta 0:00:05
   ---------- ----------------------------- 2.9/10.7 MB 2.0 MB/s eta 0:00:04
   ------------ --------------------------- 3.4/10.7 MB 2.1 MB/s eta 0:00:04
   ------------- -------------------------- 3.7/10.7 MB 2.1 MB/s eta 0:00:04
   -------------- ------------------------- 3.9/10.7 MB 2.1 MB/s eta 0:00:04
   -------------- ------------------------- 3.9/10.7 MB 2.1 MB/s eta 0:00:04
   -------------- ------------------------- 3.9/10.7 MB 2.1 MB/s eta 0:00:04
   ----------

  You can safely remove it manually.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!pip install category_encoders==2.6.3



  Attempting uninstall: category_encoders
    Found existing installation: category_encoders 2.8.1
    Uninstalling category_encoders-2.8.1:
      Successfully uninstalled category_encoders-2.8.1



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
# import all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from category_encoders import BinaryEncoder 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [22]:
import sklearn
sklearn.__version__

'1.5.1'

In [7]:
df = pd.read_csv('final_training_data (1).csv')

In [8]:
df.head()

,Complaint,Category,Subcategory,Priority
0,absence certified firstaid stations events maj...,Community Services,Community Events,Low
1,red road hall park traffic staying long malfun...,Electrical Issues,Traffic Signal Problem,Medium
2,shelters offer sufficient programs homeless in...,Community Services,Homelessness and Social Services,Low
3,public transportation system provide adequate ...,Infrastructure,Accessibility Issues,Low
4,Litter on Streets seems broken,Waste Management,Litter on Streets,Medium


In [9]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [10]:
# apply nlp to Complaint_text column
def nlp_complaint_text(text):
    sw = stopwords.words('english')
    ps = PorterStemmer()
    cm = []
    for i in range(len(text)):
        cl = re.sub('[^a-zA-Z]', ' ', text.iloc[i]
)
        cl = cl.lower()
        cl = word_tokenize(cl)
        cl = [ps.stem(word) for word in cl if word not in sw]
        cl = ' '.join(cl)
        cm.append(cl)
    return cm
        

In [11]:
x = nlp_complaint_text(df['Complaint'])

In [12]:
x[0:2]

['absenc certifi firstaid station event major safeti oversight',
 'red road hall park traffic stay long malfunct citi light highland caus inconveni sever day']

In [13]:
cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [30]:
y1, y2, y3 = df['Category'], df['Subcategory'], df['Priority']

In [31]:
# apply binary encoding in y2 and y1 and label encoder in y3
be = BinaryEncoder()
y2 = be.fit_transform(y2)
y1 = be.fit_transform(y1)
le = LabelEncoder()
y3 = le.fit_transform(y3)


In [32]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(x, y1, test_size=0.2, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(x, y2, test_size=0.2, random_state=42)
x_train3, x_test3, y_train3, y_test3 = train_test_split(x, y3, test_size=0.2, random_state=42)

In [33]:
x_train1_cv = cv.fit_transform(x_train1)
x_train2_cv = cv.fit_transform(x_train2)
x_train3_cv = cv.fit_transform(x_train3)

In [ ]:
from sklearn.model_selection import GridSearchCV

# كل النماذج + المعاملات المحتملة للتجربة
models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(solver='saga', max_iter=5000, random_state=0),
        'params': {
            'C': [0.25, 0.5, 0.75, 1],
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'l1_ratio': [None, 0.5]  # لازم مع elasticnet
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=0),
        'params': {
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [None, 5, 10]
        }
    },
    'SVM': {
        'model': SVC(probability=True),
        'params': {
            'C': [0.5, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'params': {}  # مفيش معاملات مهمة هنا
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    },
    'MLP (Neural Net)': {
        'model': MLPClassifier(max_iter=1000, random_state=0),
        'params': {
            'hidden_layer_sizes': [(50,), (100,), (50,50)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.05, 0.1]
        }
    }
}

# تجربة النماذج واحدة واحدة
results = []

for name, mp in models_params.items():
    print(f"\n🔍 Running GridSearch for: {name}")
    try:
        grid = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='accuracy', n_jobs=-1)
        grid.fit(x_train1_cv, y_train1)
        print(f"✅ Best Accuracy: {grid.best_score_:.4f}")
        print(f"🏆 Best Params: {grid.best_params_}")
        results.append((name, grid.best_score_, grid.best_params_))
    except Exception as e:
        print(f"❌ Error in {name}: {e}")

# طباعة النتائج مرتبة
results.sort(key=lambda x: x[1], reverse=True)
print("\n📊 Results (sorted):")
for name, score, params in results:
    print(f"{name}: {score:.4f} - {params}")



🔍 Running GridSearch for: Logistic Regression
❌ Error in Logistic Regression: 
All the 160 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1239, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\utils\validati

c:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan]
  self.scorer_ = scorers


✅ Best Accuracy: nan
🏆 Best Params: {'max_depth': None, 'min_samples_split': 2}

🔍 Running GridSearch for: Random Forest


c:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  self.scorer_ = scorers


✅ Best Accuracy: nan
🏆 Best Params: {'max_depth': None, 'n_estimators': 50}

🔍 Running GridSearch for: SVM
❌ Error in SVM: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hp\anaconda3\Lib\site-packages\sklearn\svm\_base.py", line 197, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "c:\Users\hp\anaconda3\Lib\site-pa

c:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  self.scorer_ = scorers


✅ Best Accuracy: nan
🏆 Best Params: {'n_neighbors': 3, 'weights': 'uniform'}

🔍 Running GridSearch for: MLP (Neural Net)


In [ ]:
from sklearn.model_selection import GridSearchCV

# كل النماذج + المعاملات المحتملة للتجربة
models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(solver='saga', max_iter=5000, random_state=0),
        'params': {
            'C': [0.25, 0.5, 0.75, 1],
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'l1_ratio': [None, 0.5]  # لازم مع elasticnet
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=0),
        'params': {
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [None, 5, 10]
        }
    },
    'SVM': {
        'model': SVC(probability=True),
        'params': {
            'C': [0.5, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'params': {}  # مفيش معاملات مهمة هنا
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    },
    'MLP (Neural Net)': {
        'model': MLPClassifier(max_iter=1000, random_state=0),
        'params': {
            'hidden_layer_sizes': [(50,), (100,), (50,50)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.05, 0.1]
        }
    }
}

# تجربة النماذج واحدة واحدة
results = []

for name, mp in models_params.items():
    print(f"\n🔍 Running GridSearch for: {name}")
    try:
        grid = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='accuracy', n_jobs=-1)
        grid.fit(x_train2_cv, y_train2)
        print(f"✅ Best Accuracy: {grid.best_score_:.4f}")
        print(f"🏆 Best Params: {grid.best_params_}")
        results.append((name, grid.best_score_, grid.best_params_))
    except Exception as e:
        print(f"❌ Error in {name}: {e}")

# طباعة النتائج مرتبة
results.sort(key=lambda x: x[1], reverse=True)
print("\n📊 Results (sorted):")
for name, score, params in results:
    print(f"{name}: {score:.4f} - {params}")


In [ ]:
from sklearn.model_selection import GridSearchCV

# كل النماذج + المعاملات المحتملة للتجربة
models_params = {
    'Logistic Regression': {
        'model': LogisticRegression(solver='saga', max_iter=5000, random_state=0),
        'params': {
            'C': [0.25, 0.5, 0.75, 1],
            'penalty': ['l1', 'l2', 'elasticnet', 'none'],
            'l1_ratio': [None, 0.5]  # لازم مع elasticnet
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=0),
        'params': {
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [None, 5, 10]
        }
    },
    'SVM': {
        'model': SVC(probability=True),
        'params': {
            'C': [0.5, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'params': {}  # مفيش معاملات مهمة هنا
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7],
            'weights': ['uniform', 'distance']
        }
    },
    'MLP (Neural Net)': {
        'model': MLPClassifier(max_iter=1000, random_state=0),
        'params': {
            'hidden_layer_sizes': [(50,), (100,), (50,50)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=0),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.05, 0.1]
        }
    }
}

# تجربة النماذج واحدة واحدة
results = []

for name, mp in models_params.items():
    print(f"\n🔍 Running GridSearch for: {name}")
    try:
        grid = GridSearchCV(mp['model'], mp['params'], cv=5, scoring='accuracy', n_jobs=-1)
        grid.fit(x_train3_cv, y_train3)
        print(f"✅ Best Accuracy: {grid.best_score_:.4f}")
        print(f"🏆 Best Params: {grid.best_params_}")
        results.append((name, grid.best_score_, grid.best_params_))
    except Exception as e:
        print(f"❌ Error in {name}: {e}")

# طباعة النتائج مرتبة
results.sort(key=lambda x: x[1], reverse=True)
print("\n📊 Results (sorted):")
for name, score, params in results:
    print(f"{name}: {score:.4f} - {params}")
